In [1]:
# imports
import time
import pandas as pd
import requests
from datetime import datetime
from riotwatcher import LolWatcher, ApiError
from pathlib import Path

# repository root: prefer file-based parent when available, else two levels up
try:
    ROOT = Path(__file__).resolve().parents[1]
except NameError:
    ROOT = Path().resolve().parents[1]


In [ ]:
watcher = None

# parse the data we need from match jsons
def parse_match_data(match_json):
    participants = match_json["info"]["participants"]
    red, blue = {}, {}
    for p in participants:
        champ = p["championName"]
        pos = p.get("teamPosition", "").lower()
        team = p["teamId"]
        if team == 100:
            blue[pos] = champ
        elif team == 200:
            red[pos] = champ
    blue_win = 1 if match_json["info"]["teams"][0]["win"] else 0
    return {
        "red_top": red.get("top"),
        "red_jg": red.get("jungle") or red.get("jg"),
        "red_mid": red.get("middle") or red.get("mid"),
        "red_adc": red.get("bottom") or red.get("adc"),
        "red_sup": red.get("utility") or red.get("support"),
        "blue_top": blue.get("top"),
        "blue_jg": blue.get("jungle") or blue.get("jg"),
        "blue_mid": blue.get("middle") or blue.get("mid"),
        "blue_adc": blue.get("bottom") or blue.get("adc"),
        "blue_sup": blue.get("utility") or blue.get("support"),
        "winner": blue_win,
    }

# fetch matches from api using watcher
def fetch_match(region, mid, timeout=8, total_timeout=120):
    global watcher
    if watcher is None:
        watcher = make_watcher()
    start = time.time()
    while time.time() - start < total_timeout:
        try:
            return watcher.match.by_id(region, mid)
        except ApiError as e:
            # key expired
            if e.response.status_code == 401:
                print("Riot API key expired. Regenerate key and save to riot_api_key.txt.")
                input("Press Enter once the key file is updated...")
                watcher = make_watcher()
                continue
            # rate limit, waits till limit is over
            elif e.response.status_code == 429:
                retry_after = int(e.response.headers.get("Retry-After", 2))
                print(f"Rate limit hit. Waiting {retry_after}s...")
                time.sleep(retry_after + 1)
                continue
            # anything else
            else:
                print(f"API error {e.response.status_code} for {mid}")
                time.sleep(1)
        except Exception as e:
            print(f"Error fetching {mid}: {e}")
            time.sleep(1)
    print(f"Skipped match {mid} after {int(time.time() - start)}s.")
    return None

# builds dataframe and writes output to files
def build_dataframe(api_key, region, puuid_list, output_file="matches_latest.csv"):
    global watcher
    watcher = make_watcher()
    all_rows, seen_matches = [], set()
    total_matches = 0

    # resolve output path relative to repo root if not absolute
    out_path = Path(output_file)
    if not out_path.is_absolute():
        # prefer the repo-level data/ryan_match_data folder for match outputs
        out_path = Path(ROOT) / "data" / "ryan_match_data" / out_path
    out_path.parent.mkdir(parents=True, exist_ok=True)

    print(f"Writing to {out_path}")

    for i, entry in enumerate(puuid_list, start=1):
        puuid = entry.get("puuid") if isinstance(entry, dict) else None
        if not puuid:
            try:
                summ = watcher.summoner.by_id(PLATFORM, entry["summonerId"])
                puuid = summ["puuid"]
            except Exception as e:
                print(f"Error getting puuid for player {i}: {e}")
                continue

        print(f"\nPlayer {i}/{len(puuid_list)}: {puuid[:10]}")
        match_ids, start_idx = [], 0

        while True:
            try:
                ids = watcher.match.matchlist_by_puuid(region, puuid, start=start_idx, count=100)
                if not ids:
                    break
                match_ids.extend(ids)
                start_idx += 100
                time.sleep(0.4)
            except ApiError as e:
                if e.response.status_code == 429:
                    retry_after = int(e.response.headers.get("Retry-After", 2))
                    print(f"Rate limit. Sleeping {retry_after}s...")
                    time.sleep(retry_after + 1)
                    continue
                print(f"Error fetching match list for {puuid[:10]}: {e}")
                break

        print(f"Retrieved {len(match_ids)} matches.")
        for j, mid in enumerate(match_ids, start=1):
            if mid in seen_matches:
                continue
            data = fetch_match(region, mid, timeout=8, total_timeout=100)
            if not data:
                continue
            try:
                row = parse_match_data(data)
                row["match_id"] = mid
                all_rows.append(row)
                seen_matches.add(mid)
                total_matches += 1
                print(f"Match {j}/{len(match_ids)} ({mid}) added ({total_matches} total).")
                if total_matches % 100 == 0:
                    pd.DataFrame(all_rows).to_csv(str(out_path), index=False)
                    print(f"Saved {total_matches} matches.")
            except Exception as e:
                print(f"Parse error for {mid}: {e}")
            time.sleep(0.4)
        time.sleep(1)

    pd.DataFrame(all_rows).to_csv(str(out_path), index=False)
    print(f"Finished. Saved {len(all_rows)} matches to {out_path}")
    return pd.DataFrame(all_rows)

# finds all top 300 players
def get_challenger_entries(region="na1", queue="RANKED_SOLO_5x5", api_key=None):
    headers = {"X-Riot-Token": api_key}
    url = f"https://{region}.api.riotgames.com/lol/league/v4/challengerleagues/by-queue/{queue}"
    r = requests.get(url, headers=headers)
    r.raise_for_status()
    return r.json().get("entries", [])

# prompts input for API key and saves it in file
def load_key():
    # prefer an API key file in the same directory as this notebook/script
    try:
        local_path = Path(__file__).resolve().parent / "riot_api_key.txt"
    except NameError:
        local_path = Path.cwd() / "riot_api_key.txt"

    if local_path.exists():
        return local_path.read_text().strip()

    # fallback to repo-level key file
    key_path = Path(ROOT) / "riot_api_key.txt"
    if key_path.exists():
        return key_path.read_text().strip()

    # if not found, prompt and save to repo-level file
    new_key = input("Enter your Riot API key: ").strip()
    key_path.write_text(new_key)
    return new_key


def make_watcher():
    return LolWatcher(load_key())

REGION = "americas"
PLATFORM = "na1"

api_key = load_key()
watcher = make_watcher()
entries = get_challenger_entries(region=PLATFORM, api_key=api_key)
print(f"Retrieved {len(entries)} Challenger entries from {PLATFORM}")
df = build_dataframe(api_key, REGION, entries, output_file=f"matches_{datetime.now().strftime('%Y-%m-%d_%H-%M-%S')}.csv")
print(df.head())

Retrieved 300 Challenger entries from na1
Writing to C:\Users\Laser\cs171-lol-draft-analysis\data\ryan_match_data\matches_2025-11-24_00-42-16.csv

Player 1/300: GchYaLfatq
Retrieved 946 matches.
Match 1/946 (NA1_5418233073) added (1 total).
Retrieved 946 matches.
Match 1/946 (NA1_5418233073) added (1 total).
Match 2/946 (NA1_5418227132) added (2 total).
Match 2/946 (NA1_5418227132) added (2 total).
Match 3/946 (NA1_5418213545) added (3 total).
Match 3/946 (NA1_5418213545) added (3 total).
Match 4/946 (NA1_5418192529) added (4 total).
Match 4/946 (NA1_5418192529) added (4 total).
Match 5/946 (NA1_5418164436) added (5 total).
Match 5/946 (NA1_5418164436) added (5 total).
Match 6/946 (NA1_5418136090) added (6 total).
Match 6/946 (NA1_5418136090) added (6 total).
Match 7/946 (NA1_5418106839) added (7 total).
Match 7/946 (NA1_5418106839) added (7 total).
Match 8/946 (NA1_5418063719) added (8 total).
Match 8/946 (NA1_5418063719) added (8 total).
Match 9/946 (NA1_5415435203) added (9 total).
